Decision Tree: Kaggle Competition for House Prices

**bold text**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import warnings
!pip install scikit-learn --upgrade
from matplotlib import pyplot as plt
%matplotlib inline
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.shape, test.shape

((1460, 81), (1459, 80))

In [ ]:
#Corelation between SalePrice and other Features
corr_matrix = train.corr()
corr_matrix['SalePrice'].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

features_matrix = ['SalePrice','OverallQual','GrLivArea','GarageCars','GarageArea','TotalBsmtSF']

scatter_matrix(train[features_matrix],figsize=(24,12))

In [ ]:
train_features = train.drop('SalePrice',axis=1)
train_labels = train['SalePrice']

In [ ]:
numeric = train_features._get_numeric_data().columns
categoric = [i for i in train_features.columns if i not in numeric]

In [ ]:
train_features.isnull().sum().index
null_columns=[]
for i in range(len(train_features.isnull().sum().index)):
    if train_features.isnull().sum()[i] !=0:
        print(train_features.isnull().sum().index[i],'-',train_features.isnull().sum()[i])
        null_columns.append(train_features.isnull().sum().index[i])

In [ ]:
null_num_columns = [i for i in null_columns if i in numeric]
null_cat_columns = [i for i in null_columns if i not in numeric]

In [ ]:
train_num = train_features[numeric]
train_cat = train_features[categoric]

In [ ]:
# Numerical Imputing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scalar',StandardScaler())
])

In [ ]:
# Categorical Imputing

from sklearn.compose import ColumnTransformer

final_pipeline = ColumnTransformer([
    ('num',num_pipeline,list(train_num.columns)),
    ('cat',OneHotEncoder(handle_unknown='ignore'),list(train_cat.columns))
])

train_final = final_pipeline.fit_transform(train_features)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtree_model = DecisionTreeRegressor()
dtree_model.fit(train_final, train_labels)

In [ ]:
def pred_rand(model):
    t_data = train_features.loc[:5]
    t_labels = train_labels.loc[:5]

    t_data_tr = final_pipeline.transform(t_data)
    pred = model.predict(t_data_tr)
    pred = [int(i) for i in pred]
    print('Predicted: ',pred)
    print('Real Labels: ',list(t_labels))

pred_rand(dtree_model)

In [ ]:
train_predictions = dtree_model.predict(train_final)
dtree_mse = mean_squared_error(train_labels, train_predictions)
dtree_rmse = np.sqrt(dtree_mse)
print(dtree_rmse)

In [ ]:
from sklearn.model_selection import cross_val_score
dtree_scores = cross_val_score(dtree_model, train_final, train_labels, scoring="neg_mean_squared_error", cv=10)
dtree_scores = np.sqrt(-dtree_scores)

In [ ]:
print('Score: ',dtree_scores)
print('Mean: ',dtree_scores.mean())
print('Standard Deviation: ',dtree_scores.std())

In [ ]:
MODEL=dtree_model.fit(train_final, train_labels)

In [ ]:
train_predictions =dtree_model.predict(train_final)

In [ ]:
test_final = final_pipeline.transform(test)

In [ ]:
pred_prices = dtree_model.predict(test_final)

In [ ]:
submissiondtree_model = pd.DataFrame({'Id': test.Id, 'SalePrice': pred_prices})
submissiondtree_model.to_csv('submission_dtree_model.csv', index=False)